In [1]:
import re
import os
import json
import pandas as pd
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import urllib 

# 解析

In [29]:
def recursive_parser(item, texts, pre_class):
    for child in item:
        try:
#             if isinstance(child, str):
#                 content = child.strip()
#                 if len(content) > 0:
#                     texts.append(content)
#                 continue

            tag = child.name
            
            content = child.text.strip()
            if len(content) == 0:
                continue
            
            if tag == 'ul' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['vitTOC']:
                    continue
            if tag == 'a' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['mediaPrmo ss']:
                    continue
            if tag == 'p' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['credits']:
                    continue
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe']: # 加视频
                continue

            if tag == 'center':
                texts.append(content)
                continue

            if tag == 'div' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['printpreview_header', 'fdaWrapper', 'adCont']:
                    continue
            
            if tag == 'section' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['sponsors']:
                    continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'otherNames', 'apPageBottom', 'contentad', 'medianet', 'from_webmd', 'toc_lite', 'monolastreviewed', 'drug_rel_main', 'disclaimer']:
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
                    recursive_parser(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol', 'article']:
                # 这几类标签 直接递归处理子标签
                recursive_parser(child, texts, '')
                
            else:
                # 剩余情况直接获取标签内容
                texts.append(content)

        except Exception as e:
            print(e)

In [30]:
def parser(input_path, output_path, tag, results):
    for root, dirs, file_names in os.walk(input_path):
        """
        os.walk()
        root 所指的是当前正在遍历的这个文件夹的本身的地址
        files_names同样是 list , 内容是该文件夹中所有的文件(不包括子目录)
        函数会自动改变root的值使得遍历所有的子文件夹。
        所以返回的三元元组的个数为所有子文件夹（包括子子文件夹，子子子文件夹等等）加上1（根文件夹）
        """
        
        for file_name in file_names:
            html_file = os.path.join(root, file_name)
            html_content = open(html_file, 'r', encoding='utf-8').read()
            selector = Selector(text=html_content)
            try:
                title = selector.xpath("//h1//text()").extract()[0].strip()
            except Exception as e:
                print(e)
                print(html_file)
                continue

            texts = []
            
            soup = BeautifulSoup(html_content, 'html.parser') 

            if tag=='supplements':
                item = soup.find('div',{'class':'w-full'})
            if tag=='drugs':
                item = soup.find('article',{'class':'pageContainer'})
            if tag=='drugs_comparasion':
                item = soup.find('article',{'id':'article_rdr'})
                if item is None:
                    item = soup.find('article',{'class':'pageContainer'})
            if tag=='drugs_class':
                item = soup.find('div',{'class':'w-full'})

            if item is None:
                print(html_file)
                continue
            else:
                recursive_parser(item, texts, "")  
                content = '\n'.join(texts)
                if content == "":
                    continue
                tmp = {
                    "title": title,
                    "content": content
                }
                results.append(tmp)

In [31]:
tags = ['supplements', 'drugs', 'drugs_comparasion', 'drugs_class']
output_path = 'D:/Intern_Project/8_RxList/'
for tag in tags: 
    results = []
    print(tag)
    source_path = f'D:/Intern_Project/8_RxList/{tag}_htmls'
    parser(source_path, output_path, tag, results)
    with open(f'D:/Intern_Project/8_RxList/{tag}_parser_results.json', 'w', encoding="utf-8") as f:
        f.write(json.dumps(results, indent=1, ensure_ascii=False))
        # f.write('\n'.join(results))

supplements
drugs
drugs_comparasion
drugs_class


In [23]:
print(len(results))

16480


In [52]:
for item in supplements_results:
    item['content'] = item['content'].replace(
        "QUESTION\nNext to red peppers, you can get the most vitamin C from ________________.","")

In [55]:
with open(f'D:/Intern_Project/8_RxList/parser_results/supplements_parser_results.json', 'w', encoding="utf-8") as f:
        f.write(json.dumps(supplements_results, indent=1, ensure_ascii=False))

In [61]:
with open('D:/Intern_Project/8_RxList/parser_results/drugs_parser_results.json', 'r', encoding='utf-8') as f:
    drugs_parser_results = json.load(f)
print(len(drugs_parser_results))

8110


In [62]:
with open('D:/Intern_Project/8_RxList/parser_results/supplements_parser_results.json', 'r', encoding='utf-8') as f:
    supplements_results = json.load(f)
print(len(supplements_results))

1193


In [63]:
with open('D:/Intern_Project/8_RxList/parser_results/drugs_comparasion_parser_results.json', 'r', encoding='utf-8') as f:
    drugs_comparasion_parser_results = json.load(f)
print(len(drugs_comparasion_parser_results))

1804


In [64]:
with open('D:/Intern_Project/8_RxList/parser_results/drugs_class_parser_results.json', 'r', encoding='utf-8') as f:
    drugs_class_parser_results = json.load(f)
print(len(drugs_class_parser_results))

750
